In [2]:
%load_ext autoreload
%autoreload 1

In [3]:
import os
import sys
import time
import numpy
import theano
import theano.tensor as T
import pandas as pd;

Using gpu device 0: GeForce GTX 980 Ti (CNMeM is disabled)


In [4]:
split = (0.7, 0.2, 0.1)

In [5]:
data = pd.read_csv('data/train.csv', header=0);
data.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
ratio_train, ratio_validate, ratio_test = split;

cols = list(data.columns.values)
cols.remove('label')

X = data.as_matrix(cols)
y = data['label'].values


rows_num = data.values.shape[0]
num_train = rows_num*ratio_train
num_validate = rows_num*ratio_validate
num_test = rows_num*ratio_test

train_set = (X[0:num_train], y[0:num_train]);
valid_set = (X[num_train:num_train+num_validate], y[num_train:num_train+num_validate]);
test_set = (X[num_train+num_validate:rows_num], y[num_train+num_validate:rows_num]);


In [7]:
#train_set, valid_set, test_set format: tuple(input, target)
#input is an numpy.ndarray of 2 dimensions (a matrix)
#witch row's correspond to an example. target is a
#numpy.ndarray of 1 dimensions (vector)) that have the same length as
#the number of rows in the input. It should give the target
#target to the example with the same index in the input.

def shared_dataset(data_xy, borrow=True):
    """ Function that loads the dataset into shared variables

    The reason we store our dataset in shared variables is to allow
    Theano to copy it into the GPU memory (when code is run on GPU).
    Since copying data into the GPU is slow, copying a minibatch everytime
    is needed (the default behaviour if the data is not in a shared
    variable) would lead to a large decrease in performance.
    """
    data_x, data_y = data_xy
    shared_x = theano.shared(numpy.asarray(data_x,
                                           dtype=theano.config.floatX),
                             borrow=borrow)
    shared_y = theano.shared(numpy.asarray(data_y,
                                           dtype=theano.config.floatX),
                             borrow=borrow)
    # When storing data on the GPU it has to be stored as floats
    # therefore we will store the labels as ``floatX`` as well
    # (``shared_y`` does exactly that). But during our computations
    # we need them as ints (we use labels as index, and if they are
    # floats it doesn't make sense) therefore instead of returning
    # ``shared_y`` we will have to cast it to int. This little hack
    # lets ous get around this issue

    #return shared_x, T.cast(shared_y, 'float32')
    return shared_x, T.cast(shared_y, 'int32')

test_set_x, test_set_y = shared_dataset(test_set)
valid_set_x, valid_set_y = shared_dataset(valid_set)
train_set_x, train_set_y = shared_dataset(train_set)

rval = [(train_set_x, train_set_y), (valid_set_x, valid_set_y),
        (test_set_x, test_set_y)]

datasets = rval
n_in = len(cols)

In [8]:
def write_bias_variance_data(filename, list1, list2, list3):    
    import csv

    with open(filename, "wb") as f:
        writer = csv.writer(f)
        csv_data = zip(list1, list2, list3)
        writer.writerows(csv_data)

In [9]:
%aimport nn_modules.CNN

hidden_layer_list = []
validate_list = []
batch_list = []


for n_batch in [1]:
    for n_hidden in [1]:
        hidden_layer_list.append(n_hidden)
        batch_list.append(n_batch)
        
        print n_batch
        
        image_size = (28, 28)
        pool_sizes = [(2, 2), (2, 2)]
        filter_sizes = [(5,5), (5,5)]
        nkerns = [20, 50]
        n_epochs = 100
    
        cnn = nn_modules.CNN.CNN(nkerns=nkerns, batch_size=100, image_size=image_size, pool_sizes=pool_sizes, filter_sizes=filter_sizes, n_epochs=n_epochs)
        validate_loss = cnn.train(datasets)
        
        validate_list.append(validate_loss)
        
        write_bias_variance_data("cnn.csv", hidden_layer_list, batch_list, validate_list)


1
lin_output :  TensorType(float32, matrix) 2
... training
training @ iter =  0
training @ iter =  100
training @ iter =  200
epoch 1, minibatch 293/293, validation error 4.095238 %
     epoch 1, minibatch 293/293, test error of best model 4.595238 %
training @ iter =  300
training @ iter =  400
training @ iter =  500
epoch 2, minibatch 293/293, validation error 3.011905 %
     epoch 2, minibatch 293/293, test error of best model 3.285714 %
training @ iter =  600
training @ iter =  700
training @ iter =  800
epoch 3, minibatch 293/293, validation error 2.380952 %
     epoch 3, minibatch 293/293, test error of best model 2.785714 %
training @ iter =  900
training @ iter =  1000
training @ iter =  1100
epoch 4, minibatch 293/293, validation error 2.297619 %
     epoch 4, minibatch 293/293, test error of best model 2.714286 %
training @ iter =  1200
training @ iter =  1300
training @ iter =  1400
epoch 5, minibatch 293/293, validation error 2.107143 %
     epoch 5, minibatch 293/293, test

The code for file CNN.py ran for 6.33m


In [27]:
print test_set[0].shape

predicted = cnn.predict(test_set[0].astype(theano.config.floatX))
label = test_set[1]


s=0
f=0
for p, l in zip(predicted, label):
    if p == l:
        s += 1
    else:
        f += 1
        
print s
print f
print float(s)/(s+f)*100,
print '%'

(4200L, 784L)
4147
53
98.7380952381 %


In [42]:
data = pd.read_csv('data/test.csv', header=0);
predicted = cnn.predict(data.astype(theano.config.floatX))


In [65]:
df = pd.DataFrame([numpy.arange(1, data.shape[0]+1), predicted]).T
df.columns = ['ImageId', 'Label']

In [66]:
df.head()

,ImageId,Label
0,1,2
1,2,0
2,3,9
3,4,9
4,5,3


In [67]:
df.to_csv("result.csv", cols=['ImageId', 'Label'], index=False, encoding='utf-8')

In [ ]:
image_size = (28, 28)
pool_sizes = [(2, 2), (2, 2)]
filter_sizes = [(5,5), (5,5)]
nkerns = [20, 50]
n_epochs = 100

cnn = nn_modules.CNN.CNN(nkerns=nkerns, batch_size=100, image_size=image_size, pool_sizes=pool_sizes, filter_sizes=filter_sizes, n_epochs=n_epochs)

data = pd.read_csv('data/test.csv', header=0);
predicted = cnn.predict(data.astype(theano.config.floatX))

df = pd.DataFrame([numpy.arange(1, data.shape[0]+1), predicted]).T
df.columns = ['ImageId', 'Label']

df.to_csv("result.csv", cols=['ImageId', 'Label'], index=False, encoding='utf-8')

In [ ]:
import numpy
import pylab
from PIL import Image
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

# open random image of dimensions 639x516
img = Image.open(open('data/5_684.bmp'))
#img = numpy.asarray(img, dtype='float64') / 256.
img = numpy.asarray(img, dtype='float')

_img = img.reshape(1, 784)
_img = numpy.apply_over_axes((lambda x,axis:x>0), _img, [0])

f = theano.function([cnn.x], cnn.layers[0][0].output)

# put image in 4D tensor of shape (1, 3, height, width)
filtered_img = f(_img)

# plot original image and first and second components of output
pylab.subplot(1, 3, 1); pylab.axis('off'); pylab.imshow(img)
pylab.gray();
# recall that the convOp output (filtered image) is actually a "minibatch",
# of size 1 here, so we take index 0 in the first dimension:
pylab.subplot(1, 3, 2); pylab.axis('off'); pylab.imshow(filtered_img[0, 0, :, :])
pylab.subplot(1, 3, 3); pylab.axis('off'); pylab.imshow(filtered_img[0, 1, :, :])
pylab.show()